## Visualizing crime statistics in LA

### Goal: 
To understand the crime landscape in LA by answering the following questions.
#### What are most significant crimes in LA?
#### Is the police response a match for the criminal activity?
#### What recommendations would you give to someone planning to visit LA soon?

In [1]:
import pandas as pd


In [3]:
path = 'los-angeles-crime-arrest-data/'
cri = pd.read_csv(path + 'crime-data-from-2010-to-present.csv', encoding='utf-8')
arr = pd.read_csv(path + 'arrest-data-from-2010-to-present.csv', encoding='utf-8')

In [28]:
cri['Weapon Description'].value_counts() /cri['Weapon Description'].value_counts().sum()

STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    0.606357
VERBAL THREAT                                     0.082774
UNKNOWN WEAPON/OTHER WEAPON                       0.080322
HAND GUN                                          0.048087
SEMI-AUTOMATIC PISTOL                             0.018446
KNIFE WITH BLADE 6INCHES OR LESS                  0.018055
OTHER KNIFE                                       0.013623
UNKNOWN FIREARM                                   0.011177
VEHICLE                                           0.010245
REVOLVER                                          0.008090
BOTTLE                                            0.007140
STICK                                             0.006420
ROCK/THROWN OBJECT                                0.006418
CLUB/BAT                                          0.006140
BLUNT INSTRUMENT                                  0.006110
FOLDING KNIFE                                     0.005994
MACE/PEPPER SPRAY                                 0.0054

In [32]:
cri.columns

Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code', 'Premise Description',
       'Weapon Used Code', 'Weapon Description', 'Status Code',
       'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3',
       'Crime Code 4', 'Address', 'Cross Street', 'Location '],
      dtype='object')

In [36]:
cri['Status Description'].value_counts() /cri['Status Description'].value_counts().sum()

Invest Cont     0.772084
Adult Other     0.115213
Adult Arrest    0.102603
Juv Arrest      0.007622
Juv Other       0.002462
UNK             0.000016
Name: Status Description, dtype: float64

In [33]:
cri[[ 'Victim Descent', 'Premise Code', 'Premise Description',
       'Weapon Used Code', 'Weapon Description', 'Status Code',
       'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3',
       'Crime Code 4']].head()

,Victim Descent,Premise Code,Premise Description,Weapon Used Code,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4
0,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN
1,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN
2,NaN,108.0,PARKING LOT,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN
3,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN
4,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN


In [18]:
cri['Crime Code Description'].value_counts().head() / float(cri['Crime Code Description'].value_counts().sum())


AttributeError: 'int' object has no attribute 'head'

In [23]:
cri['Crime Code Description'].value_counts().head(10)

BATTERY - SIMPLE ASSAULT                                   180434
BURGLARY FROM VEHICLE                                      153451
VEHICLE - STOLEN                                           151622
THEFT PLAIN - PETTY ($950 & UNDER)                         141489
BURGLARY                                                   140926
THEFT OF IDENTITY                                          120835
INTIMATE PARTNER - SIMPLE ASSAULT                          107900
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)    102589
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT              86829
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                    86440
Name: Crime Code Description, dtype: int64

In [22]:
arr['Charge Description'].value_counts().head(10)

DRUNK DRIVING ALCOHOL/DRUGS                 95410
DRINKING IN PUBLIC                          92965
CORPORAL INJURY ON SPOUSE/COHABITANT/ETC    44812
POSSESSION CONTROLLED SUBSTANCE             43697
FTA AFTER WRITTEN PROMISE                   31177
POSSESSION NARCOTIC CONTROLLED SUBSTANCE    29352
PROSTITUTION                                26169
OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE     25668
SIT/LIE/SLEEP SIDEWALK OR STREET            25573
LOS ANGELES MUNICIPAL CODE                  24367
Name: Charge Description, dtype: int64

In [29]:
arr.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location',
       'Zip Codes', 'Census Tracts', 'Precinct Boundaries',
       'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)'],
      dtype='object')

In [31]:
arr[['Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description','LA Specific Plans']].head()

,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,LA Specific Plans
0,M,W,24.0,Miscellaneous Other Violations,M,594(A)PC,VANDALISM,10.0
1,M,O,NaN,NaN,O,300(B)WIC,NaN,NaN
2,F,O,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,17.0
3,F,W,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,NaN
4,M,W,8.0,Other Assaults,F,69PC,OBSTRUCT/RESIST EXECUTIVE OFFICER,10.0
